In [1]:
import kagglehub


path = kagglehub.dataset_download("adrianvinueza/gym-customers-features-and-churn")

print("Path to dataset files:", path)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|█████████████████████████████████████████| 133k/133k [00:00<00:00, 811kB/s]

Extracting files...
Path to dataset files: /Users/korneliakozaczewska/.cache/kagglehub/datasets/adrianvinueza/gym-customers-features-and-churn/versions/1
